# Notebook #1: Extracción y Transformación

- En este notebook realizaremos extraeremos datos haciendo uso de APIs, para las sigueintes fuentes de información:
1. API Geoapify: datos geográficos de una ciudad.
2. API Idealista: datos de viviendas en venta y alquiler.

- Usando este mismo notebook, se realizará la limpieza/transformación de los datos, que se cargarán posteriormente en a una base de datos. Todas las funciones aquí utilizadas encuentran su soporte en `../src/soporte_ETL.py`

- Dado que los datos se modifican cada vez que se ejecuta una función de consulta, las mismas están comentadas. Para ejecutarlas, debe eliminarse la #, con la consecuencia de que, los datos de origen serán sustituidos.

- El primer paso será importar las librerías necesarias:

In [2]:
# Librerías para tratamiento de datos

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np
import re
from shapely.geometry import MultiPolygon, Polygon

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_ETL as setl #Archivo .py con funciones
#from src import soporte_variables as sv

# Librería para acceder a funcionalidades del sistema operativo
import os

# Librerías para trabajar con distintos formatos de archivos
import pickle
import json

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

- Por medio de la API de Geoapify, se extrae la información geográfica de las divisiones administrativas de Zaragoza, en este caso, los distritos.
- Una vez extraída, se guarda en un archivo json.

In [ ]:
response_distritos = setl.geoconsulta_distritos("516ca3dd0b861fedbf594631421e8bd84440f00101f9018c46050000000000c002069203085a617261676f7a61")
#with open("../data/raw/distritos_zaragoza.json", "w") as f:
#    json.dump(response_distritos, f, indent=4)

- Carga del json genrado:

In [3]:
with open("../data/raw/distritos_zaragoza.json", "r") as f:
    response_distritos = json.load(f)

## Creación del DF Distritos

- Crearemos ahora el JSON en un GeoDataFrame con los datos geográficos de cada uno de los 16 distritos, que convertiremos a geografía de polígono.
- Como paso final, guardamos el archivo.

In [4]:
data_distritos = []
for distrito in response_distritos["features"]:
    district_name = distrito["properties"].get("district", "Desconocido")
    coordinates = distrito["geometry"].get("coordinates", [])
    
    # Convertir coordenadas a MultiPolygon
    if distrito["geometry"]["type"] == "MultiPolygon":
        geometry = MultiPolygon([Polygon(polygon[0]) for polygon in coordinates])
    elif distrito["geometry"]["type"] == "Polygon":
        geometry = MultiPolygon([Polygon(coordinates[0])])
    else:
        geometry = None
    
    data_distritos.append({"distrito": district_name, "geometry": geometry})

gdf_distritos = gpd.GeoDataFrame(data_distritos, geometry="geometry", crs="EPSG:4326")

#gdf_distritos.to_file("../data/transformed/gdf_distritos.geojson", driver="GeoJSON")

- Comprobamos la apariencia del nuevo GDF y los tipos de datos.

In [6]:
gdf_distritos.head(5)

,distrito,geometry
0,Distrito Rural,"MULTIPOLYGON (((-0.9212 41.50517, -0.91835 41...."
1,Sur,"MULTIPOLYGON (((-1.06057 41.61549, -1.05914 41..."
2,Miralbueno,"MULTIPOLYGON (((-0.98545 41.65679, -0.98502 41..."
3,Santa Isabel,"MULTIPOLYGON (((-0.8438 41.67621, -0.84371 41...."
4,El Rabal,"MULTIPOLYGON (((-0.88153 41.65838, -0.87771 41..."


In [7]:
gdf_distritos.dtypes

distrito      object
geometry    geometry
dtype: object

## API Idealista

- Obtendremos datos de viviendas en alquiler a través de una API de idealista.
- De la consulta, que realizamos con la función `sf.consulta_idealista()`, obtendremos una lista correspondiente a cada página de respuesta.
- La función recibe como argumentos el tipo de operación, el código y el nombre de la ciudad que deseamos consultar y el número de páginas deseado, devolviendo 40 resultados por página. La lista de resultados la almacenaremos en un archivo de tipo `json`.
- Dadas las limitaciones de la función cada iteración tarda unos 10 segundos.

In [ ]:
resultados_idealista = setl.consulta_idealista("sale", "0-EU-ES-28-07-001-079", "Zaragoza", 10)

In [ ]:
#with open("../datos/origen/idealista.json", "w") as json_file:
#    json.dump(resultados_idealista, json_file, indent=4)

- Ahora importaremos el resultado de la consulta, comprobaremos que la longitud de la lista sea correcta y le aplicaremos la función `dataframe_idealista()`, que recibe como argumento el archivo json con los resultados y devuelve un dataframe con los datos transformados.

In [5]:
with open("../datos/origen/idealista.json", 'r') as file:
    resultados_idealista = json.load(file)

In [15]:
len(resultados_idealista)

10

In [6]:
df_idealista = sf.dataframe_idealista(resultados_idealista)

In [10]:
df_idealista["Precio"].describe()

count     400.000000
mean     2151.742500
std      1434.066231
min       700.000000
25%      1250.000000
50%      1600.000000
75%      2500.000000
max      9500.000000
Name: Precio, dtype: float64

In [12]:
df_idealista.shape

(400, 10)

- En una exploración inicial de los datos, hemos identificado una serie de anuncios con un precio que se aleja mucho de la media.
- El método IQR (Interquartile Range) es una técnica estadística utilizada para identificar y gestionar outliers (valores atípicos) en un conjunto de datos. El rango intercuartílico (IQR) es la diferencia entre el tercer cuartil (Q3) y el primer cuartil (Q1) de un conjunto de datos ordenados. IQR = Q3 - Q1. Este rango mide la dispersión de la mitad central de los datos, evitando así la influencia de los valores extremos. Los valores que caen fuera de estos límites se consideran outliers.
- En nuestro caso, utilizando la función `sf.identificar_outliers()`, que recibe como parámetros el DF y la columna que deseamos revisar, y devuelve un DF con los outliers, los identificaremos (según el describe anterior) y eliminaremos de los datos.

In [8]:
sf.identificar_outliers(df_idealista,"Precio")

,Latitud,Longitud,Precio,Tipo,Planta,Tamanio,Habitaciones,Banios,Direccion,Descripcion
7,40.436160,-3.693369,5900.0,departamento,2,130.0,3,3,"Calle De Zurbano, 61",¡Descubre Este Reformado Y Lujoso Apartamento ...
9,40.415596,-3.708451,4800.0,departamento,2,154.0,2,2,Calle De La Lechuga,S C A L A | Properties Presenta Esta Viviend...
19,40.457106,-3.683056,8000.0,departamento,None,287.0,5,4,Paseo De La Habana,Fantástico Y Amplio Piso Con Terraza Y Piscina...
23,40.418827,-3.699927,4500.0,departamento,1,133.0,2,2,"Calle De La Virgen De Los Peligros, 9",Esta Vivienda A Estrenar Destaca Por Su Calida...
47,40.448310,-3.669911,4700.0,departamento,6,170.0,3,3,Calle De López De Hoyos,Crownston Presenta Este Elegantísimo Ático Dúp...
49,40.456418,-3.631459,4500.0,chalet,None,324.0,5,5,Barrio Conde Orgaz-Piovera,Quartiers Expertos Inmobiliarios. Agents Immob...
51,40.448954,-3.669955,5000.0,chalet,None,180.0,3,2,Barrio Ciudad Jardín,Chalet Reformado De Lujo En Ciudad Jardín Magn...
61,40.410255,-3.676674,5500.0,departamento,5,238.0,4,4,Barrio Niño Jesús,Luminoso Piso Remodelado Con Vistas Al Retiro ...
77,40.472115,-3.780787,9500.0,chalet,None,1000.0,6,7,Calle Loma De Los Bailanderos,Evernest Presenta Esta Maravillosa Propiedad U...
81,40.436833,-3.685469,4900.0,ático,8,170.0,2,2,Lagasca,Diplomatic Real Estate Alquila Espectacular Át...


- En este caso, eliminaremos 32 entradas. Calculamos nuestros límites inferiores y superiores y aplicamos el filtro.

In [15]:
Q1 = df_idealista["Precio"].quantile(0.25)
Q3 = df_idealista["Precio"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lower_bound)
print(upper_bound)

-625.0
4375.0


In [18]:
df_idealista = df_idealista[(df_idealista['Precio'] <= upper_bound)]

- Eliminamos tildes en los nombres de las columnas y usamos nuevamente nuestra función de traducción sobre la columna "Tipo", obteniendo como resultado el DF que se imprime, que contiene información de 367 viviendas.

In [7]:
df_idealista["Tipo"] = df_idealista["Tipo"].apply(sf.traducir_es)
df_idealista["Direccion"] = df_idealista["Direccion"].str.title()
df_idealista["Descripcion"] = df_idealista["Descripcion"].str.title()

In [19]:
df_idealista.shape

(368, 10)

In [ ]:
df_idealista.head(2)

,Latitud,Longitud,Precio,Tipo,Planta,Tamaño,Habitaciones,Baños,Dirección,Descripción
0,40.432201,-3.714278,1350.0,departamento,1,73.0,2,1,"Calle De Andres Mellado, 18",Piso Con Una Habitación Con Cama De Matrimonio...
1,40.414669,-3.705414,2300.0,departamento,5,95.0,2,2,Calle Del Marqués Viudo De Pontejos,S C A L A | Properties Presenta Ático Amuebl...


- Cuando un valor se repite muchas veces, como puede ser el caso del tipo de vivienda, para crear nuestra base de datos, lo correcto sería obtener una nueva tabla con identificadores únicos para cada tipo de vivienda, así, normalizamos los datos, eliminando redundancias. Por una limitación de tiempo, no se hará en este proyecto.

In [20]:
df_idealista["Tipo"].value_counts()

Tipo
departamento    296
estudio          37
ático            23
dúplex           11
chalet            1
Name: count, dtype: int64

- Para poder asignar las coordenadas de cada vivienda a un distrito, transformamos la latitud y la longitud de entrada a un geopunto, y uniremos el nuevo GDF con nuestro GDF de distritos. Asignamos el tipo de CRS y guardamos en un archivo de tipo `geojson`.

In [ ]:
gdf_idealista = gpd.GeoDataFrame(df_idealista, geometry=gpd.points_from_xy(df_idealista.Longitud, df_idealista.Latitud))
gdf_idealista.crs = "EPSG:4326"
#gdf_idealista.to_file('../datos/origen/idealista.geojson', driver='GeoJSON')

In [ ]:
gdf_sjoin2 = gpd.sjoin(gdf_idealista, gdf_distritos, how="inner", predicate="within")
gdf_sjoin2 = gdf_sjoin2.drop(columns="index_right")

- Realizamos algunas tareas más de limpieza, asignación de tipos de datos y guardamos el resultado final en un archivo `geojson`.

In [23]:
gdf_sjoin2_final = gdf_sjoin2[['ID_Distrito', 'Precio', 'Tipo', 'Planta', 'Tamanio',
       'Habitaciones', 'Banios', 'Direccion', 'Descripcion', 'geometry']]
gdf_sjoin2_final["Tamanio"] = gdf_sjoin2_final["Tamanio"].astype(int)

In [24]:
gdf_sjoin2_final.sample(2)

,ID_Distrito,Precio,Tipo,Planta,Tamanio,Habitaciones,Banios,Direccion,Descripcion,geometry
236,9,1600.0,departamento,1,50,1,1,"Plaza De Alcira, 15",Disponible En: Noviembre. Reserve En Línea Ha...,POINT (-3.71428 40.46385)
113,9,2500.0,departamento,bj,142,3,2,Avenida Del Talgo,¡Descubre Tu Nuevo Hogar En Madrid! Este Impre...,POINT (-3.77726 40.44813)


In [25]:
gdf_sjoin2_final.dtypes

ID_Distrito        int32
Precio           float64
Tipo              object
Planta            object
Tamanio            int64
Habitaciones       int64
Banios             int64
Direccion         object
Descripcion       object
geometry        geometry
dtype: object

In [ ]:
#gdf_sjoin2_final.to_file('../datos/finales/idealista.geojson', driver='GeoJSON')

- Verificamos la descarga y haciendo uso de la libería `os` renombraremos el archivo.

In [9]:
os.listdir("../data/raw/")

['distritos_zaragoza.json', 'ejemplos_venta_id.pkl']

In [ ]:
#os.rename("../datos/origen/Datos de la serie 0307010000022.csv", "../datos/origen/extranjeros_madrid.csv")

- Ahora debemos importarlo a nuestro notebook y convertirlo en un dataframe.

- Sobre este, realizaremos tareas de transformación.

- Con este nuevo formato, comprobamos los tipos de datos y exportamos a un archivo CSV.

- Con esta tarea finalizamos la extracción y transformación de los datos. Continuaremos en el notebook #2 con la creación e inserción de los datos a una base de datos SQL.